In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rb-titanic/train.csv
/kaggle/input/rb-titanic/test.csv
/kaggle/input/titanic/test.csv


In [17]:
import pandas as pd

df = pd.read_csv("/kaggle/input/rb-titanic/train.csv")
print(df)
print(df.info())
print("\nDescriptive statistics:")
print(df.describe())

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())


     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [29]:

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
#import polars as pl
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, FunctionTransformer, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Notes: PCA didn't help!?
####
# NEXT STEPS: internally split the data into a sub-test so we can iterate faster!
# MAKE SUBMISSION A SECOND SECTION?

version = 12
name = "pca_95_sibs_parents"

FINAL_TEST = True
PCA_PERCENT = 95  # EXPERIMENTS = range(50, 76, 1)
if FINAL_TEST:
    EXPERIMENTS = [PCA_PERCENT] if FINAL_TEST else EXPERIMENTS
else:
    EXPERIMENTS = range(50, 99, 2)

#DROP_FEATURES = ["PassengerId", "Survived", "Pclass", "Name", "Ticket", "Fare", "Cabin"]
LABEL = ["Survived"]
SCALED_FEATURES = ["Age", "SibSp", "Parch"]  # Fare did not seem to improve
CAT_FEATURES = ["Embarked", "Pclass"]
BOOL_FEATURES = ["Sex"]

def make_pipeline(pca_percent=PCA_PERCENT):
    print(pca_percent)
    # Create the preprocessing steps for specific column processing
    preprocessor = ColumnTransformer(
        transformers=[
            (
                'num',
                Pipeline(steps=
                    [
                        ('average', SimpleImputer(missing_values=np.nan, strategy='mean')),
                        ('scale', StandardScaler()),
                    ],
                ),
                 SCALED_FEATURES,
            ),
            (
                'cat',
                Pipeline(steps=
                    [
                        ('imputer', SimpleImputer(strategy='most_frequent')),
                        ('onehot', OneHotEncoder(handle_unknown='ignore', drop="first", sparse_output=False))
                    ],
                ),
                CAT_FEATURES + BOOL_FEATURES,
            ),
        ],
        remainder="drop",
    )

    # Create the full pipeline
    pipeline = Pipeline([
        ('preprocess', preprocessor),
        ('pca', PCA(n_components=pca_percent/100)),
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42)),
    ])
    
    return pipeline

# Get train/test features and labels
df = pd.read_csv("/kaggle/input/rb-titanic/train.csv")
X = df.drop(LABEL, axis=1)
y = df[LABEL]
y = y.values.ravel()
if FINAL_TEST:
    X_train = X
    y_train = y
    X_test = pd.read_csv("/kaggle/input/rb-titanic/test.csv")
    # NOTE: There is no y_test in the final
else:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

for experiment in EXPERIMENTS:
    # Train with pipeline
    pipeline = make_pipeline(experiment)
    pipeline.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = pipeline.predict(X_test)

    # Evaluate the model
    if not FINAL_TEST:
        print(f"{version}_{name} SCORES")
        mse = mean_squared_error(y_test, y_pred)  # <-- NO y_test/Survived IN DATA. Submit?
        rmse = np.sqrt(mse)
        print(f"RMSE:\t{rmse}")
        r2 = r2_score(y_test, y_pred)
        print(f"R2:\t{r2}")
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Acc:\t{accuracy}")
        from sklearn.metrics import f1_score
        f1 = f1_score(y_test, y_pred)
        print(f"F1:\t{f1}")
        print("---")

    # Save the final test results
    else:
        # Save the DataFrame to a CSV file
        results_df = pd.DataFrame({
            "PassengerId": X_test["PassengerId"],
            "Survived": y_pred,
        })
        print(results_df)

        csv_filename = f"titanic_restart_{version}_{name}.csv"
        results_df.to_csv(csv_filename, index=False)
        print(f"Predictions saved to {csv_filename}")


95
     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         1
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]
Predictions saved to titanic_restart_11_pca_95_fare.csv
